# Trabajo Práctico 4
Encuesta Permanente de Hogares (EPH) 

Integrantes: Regina Zasiekin Drittanti, Mila Palacios Vieira y Milagros Arjona

# Limpieza de Base TP 3



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib.ticker import PercentFormatter
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, RocCurveDisplay
from sklearn.linear_model import LogisticRegression


In [ ]:
base_04_inicial = pd.read_stata('/Users/usuario/Downloads/Individual_t104(1).dta')  
base_24_inicial = pd.read_excel('/Users/usuario/Downloads/usu_individual_T124(3).xlsx')

Eliminen todas las observaciones que no corresponden a los aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos Aires, y unan ambos trimestres en una sola base. 

In [ ]:
# me quedo solo con los valores de CABA y GBA
base_04_filtrada = base_04_inicial.loc[base_04_inicial['aglomerado'].isin(['Ciudad de Buenos Aires', 'Partidos del GBA'])]
base_24_filtrada = base_24_inicial.loc[base_24_inicial['AGLOMERADO'].isin([32, 33])]

#Para poder unir sin problema, me aseguro que als variables esten en el mismo formato
base_04_filtrada.columns = base_04_filtrada.columns.str.lower()
base_24_filtrada.columns = base_24_filtrada.columns.str.lower()

# concateno las bases
union_eph = pd.concat([base_24_filtrada, base_04_filtrada])
print(union_eph.columns.tolist())

In [ ]:
# como hay datos que tienen distintas etiquetas en las dos bases que concatenamos, tenemos que unificar renombrar las etiquetas de una de las bases asi se pueden tener todos los datos con el mismo valor
variables_interes = union_eph[["ch04", "ch06", "ch07", "ch08", "nivel_ed", "estado", "cat_inac"]]

for i in variables_interes:
    if i == "ch04": 
        union_eph['ch04'] = union_eph['ch04'].replace({
            'Mujer': 2,
            'Varón': 1})
    elif i == "ch06":
        union_eph['ch06'] = union_eph['ch06'].replace({
            '98 y más años' : 98, 
            'Menos de 1 año' : 0})
        # cambio el formato de los valores de esta columna porque la mitad estaba en float y la mitas en int
        union_eph['ch06'] = union_eph['ch06'].astype(int)
    elif i == "ch08":
        union_eph['ch08'] = union_eph['ch08'].replace({
            'Obra social (incluye PAMI)': 1,  
            'Mutual/Prepaga/Servicio de emergencia': 2,  
            'Planes y seguros públicos': 3,  
            'No paga ni le descuentan': 4,  
            'Ns./Nr.': 9,  
            'Obra social y mutual/prepaga/servicio de emergencia': 12,  
            'Obra social y planes y seguros públicos': 13,  
            'Mutual/prepaga/servicio de emergencia/planes y seguros públi': 23,
            'Obra Social, mutual/prepaga/servicio de emergencia y planes y seguros públicos': 123})
    elif i == "nivel_ed":
        union_eph['nivel_ed'] = union_eph['nivel_ed'].replace({
            1: 'Primaria Incompleta (incluye educación especial)',  
            2: 'Primaria Completa',  
            3: 'Secundaria Incompleta',  
            4: 'Secundaria Completa',  
            5: 'Superior Universitaria Incompleta',  
            6: 'Superior Universitaria Completa', 
            7: 'Sin instrucción',  
            9: 'Ns./ Nr. '})

    conteo = union_eph.groupby(i).size()

In [ ]:
# Diccionario para mapear los valores originales de ch04 (Género) con nombres descriptivos
map_ch04 = {
    1: 'Masculino',
    2: 'Femenino'
}


map_ch07 = {
    1: 'Unido',
    2: 'Casado',
    3: 'Separado o divorciado',
    4: 'Viudo',
    5: 'Soltero'
}

map_estado = {
    1: 'Ocupado',
    2: 'Desocupado',
    3: 'Inactivo',
    4: 'Menor de 10 años',
    0: 'Entrevista individual no realizada (no respuesta al cuestion'
}

map_cat_inac = {
    1: 'Jubilado/pensionado',
    2: 'Rentista',
    3: 'Estudiante',
    4: 'Ama de casa',
    5: 'Menor de 6 años',
    6: 'Discapacitado',
    7: 'Otros'
}

# Reemplazar valores en las columnas antes de generar dummies
union_eph['ch04'] = union_eph['ch04'].replace(map_ch04)
union_eph['ch07'] = union_eph['ch07'].replace(map_ch07)
union_eph['estado'] = union_eph['estado'].replace(map_estado)
union_eph['cat_inac'] = union_eph['cat_inac'].replace(map_cat_inac)

# Crear variables dummies con nombres descriptivos
union_eph = pd.get_dummies(union_eph, columns=['ch04','ch07', 'estado', 'cat_inac'], drop_first=False)

# Mostrar las primeras filas para verificar los nombres descriptivos
print("Variables dummies con nombres descriptivos:")
print(union_eph.head().to_string(index=False))

In [ ]:
# Seleccionar las columnas dummies recién creadas
dummies_cols = [col for col in union_eph.columns if 'ch07_' in col or 'estado_' in col or 'cat_inac_' in col]

# Convertir las dummies de booleanos a enteros
union_eph[dummies_cols] = union_eph[dummies_cols].astype(int)

# Verificar que las dummies están en formato 1 y 0
print(union_eph[dummies_cols].head())

observaciones con valores que no tienen sentido, descártenlas (por ejemplo, ingresos y edades negativos). Expliquen las decisiones tomadas. 

In [ ]:
# Verificación de observaciones negativas en EDAD

valores_negativos_edad = union_eph['ch06'] < 0
# cantidad de datos negativos
cantidad_negativos_edad = valores_negativos_edad.sum()
print("Cantidad de valores negativos en variable edad:", cantidad_negativos_edad)

# me quedo con los valores mayores o iguales a 0 de edad
base_limpia = union_eph[union_eph['ch06'] >= 0]

# Verificación de observaciones negativas en IPCF
valores_negativos_ipcf = union_eph[union_eph['ipcf'] < 0]
# cantidad de datos negativos
cantidad_negativos_ipcf = len(valores_negativos_ipcf)
# Mostrar resultados
print("Cantidad de valores negativos en variable ipcf:", cantidad_negativos_ipcf)

# me quedo con los valores mayores o iguales a 0. Como no hay valores negativos, no saca ninguno
base_limpia = union_eph[union_eph['ipcf'] >= 0]

1. Exploren el diseño de registro de la base de hogar: a priori, ¿qué variables creen pueden ser predictivas de la desocupación y seria útil
incluir para perfeccionar el ejercicio del TP3? Mencionen estas variables y justifiquen su elección.

In [ ]:
# Se respondió en el informe 

2. Descarguen la base de microdatos de la EPH correspondiente al primer trimestre de 2004 y 2024 en formato .dta y .xls, respectivamente. La
base de hogares se llama Hogar_t104.dta y usu_hogar_T124.xls, respectivamente. Eliminen todas las observaciones que no corresponden a los aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos Aires y unan ambos trimestres en una sola base. Esto es, a la base de la encuesta individual de cada año (que usaron en el TP3) unan la base de la encuesta de hogar. Asegúrese de estar usando las variables CODUSU y NRO_Hogar para el merge.

In [3]:
# Completar

3. Limpien la base de datos tomando criterios que hagan sentido. Explicar cualquier decisión como el tratamiento de valores faltantes (missing
values), extremos (outliers), o variables categóricas. Justifique sus decisiones.

4. Construya variables (mínimo 3) que no estén en la base pero que sean relevantes para predecir individuos desocupados (por ejemplo, la
proporción de personas que trabajan en el hogar).

5. Presenten estadísticas descriptivas de tres variables de la encuesta de hogar que ustedes creen que pueden ser relevantes para predecir la
desocupación. Comenten las estadísticas obtenidas.

# Parte II: Clasificación y regularización

1. Para cada año, partan la base respondieron en una base de prueba y una de entrenamiento (X_train, y_train, X_test, y_test) utilizando
el comando train_test_split. La base de entrenamiento debe comprender el 70% de los datos, y la semilla a utilizar (random state
instance) debe ser 101. Establezca a desocupado como su variable dependiente en la base de entrenamiento (vector y). El resto de las
variables serán las variables independientes (matriz X). Recuerden agregar la columna de unos (1).

2. Expliquen brevemente cómo elegirían λ por validación cruzada (en Python es alpha). Detallen por qué no usarían el conjunto de prueba
(test) para su elección.

3. En validación cruzada, ¿cuáles son las implicancias de usar un k muy pequeño o uno muy grande? Cuando k = n (con n el número de
muestras), ¿cuántas veces se estima el modelo?

4. Para regresión logística, implementen la penalidad, L1 como la de LASSO y L2 como la de Ridge con λ = 1 (como en la Tutorial 10), usando
la opción penalty y reporten la matriz de confusión, la curva ROC, los valores de AUC y de Accuracy para cada año.1 ¿Cómo cambiaron los
resultados con respecto al TP3? ¿La performance de regresión logística con regularización es mejor o peor?

Object `modelo` not found.


5. Realicen un barrido en = 10n λ con n ∈ {−5, −4, −3 ..., +4, +5} y utilicen 10-fold CV para elegir el λ óptimo en regresión logística con Ridge y con
LASSO. ¿Qué λ seleccionó en cada caso? Usando la librería de seaborn, generen box plot mostrando la distribución del error de predicción para
cada λ. Cada box debe corresponder a un valor de λ y contener como observaciones el error medio de validación (MSE) para cada partición.
Además, para la regularización LASSO, generen un line plot del promedio de la proporción de variables ignoradas por el modelo en
función de λ (como vieron en el tutorial 10), es decir la proporción de variables para las cuales el coeficiente asociado es cero.2

6. En el caso del valor óptimo de λ para LASSO encontrado en el inciso anterior, ¿qué variables fueron descartadas? ¿Son las que hubieran
esperado? ¿Tiene relación con lo que respondieron en el inciso 1 de la Parte I?

7. Elijan alguno de los modelos de regresión logística donde hayan probado distintos parámetros de regularización y comenten: Compare los
resultados de 2004 versus 2024, ¿qué método de regularización funcionó mejor: Ridge o LASSO? ¿LASSO hizo una selección distinta de
predictores en 2004 versus 2024? Comenten mencionando el error cuadrático medio (MSE).